In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [2]:
mask = np.ones((700, 700))*255
mask = mask.astype('uint8')
prevx,prevy=0,0
curr_tool="select tool"
max_x_tool,max_y_tool=620,80
min_x_color,min_y_color=600,110
max_x_color,max_y_color=650,590
left_margin=80
curr_color=()
time_intialised=True
circle_rad=40
mode=False

In [3]:
def get_tool(x):
    
    if 80 < x < 170:
        return "circle"
    
    elif 170 < x < 260:
        return "rectangle"
    
    elif 260 < x < 350:
        return "line"
    
    elif 350 < x < 440:
        return "draw"
    
    elif 440 < x < 530:
        return "erase"
    
    elif 530 < x < 620:
        return "clear"

In [4]:
def get_color(y):
    
    if 110 < y < 190:
        return (0,0,255)
    
    elif 190 < y < 270:
        return (255,0,0)
    
    elif 270 < y < 350:
        return (0,255,0)
    
    elif 350 < y < 430:
        return (0,255,255)
    
    elif 430 < y < 510:
        return (255,255,255)
    
    elif 510 < y < 590:
        return (0,0,0)

In [5]:
def color_name(curr_color):
    
    if curr_color == (0,0,0):
        return "black"
    
    elif curr_color == (255,255,255):
        return "white"
    
    elif curr_color == (255,0,0):
        return "blue"
    
    elif curr_color == (0,255,255):
        return "yellow"
    
    elif curr_color == (0,255,0):
        return "green"
    
    elif curr_color == (0,0,255):
        return "red"
    
    elif curr_color == ():
        return "black"

In [6]:
def index_raised(yi,y9):
    if (y9-yi)>50:
        return True
    return False

In [8]:
hands = mp.solutions.hands
hand_detect = hands.Hands(static_image_mode=False,min_detection_confidence=0.66, min_tracking_confidence=0.66, max_num_hands=1)
draw = mp.solutions.drawing_utils

cap=cv2.VideoCapture(0)

while cap.isOpened():
    r,f=cap.read()
    f=cv2.resize(f,(700,700))
    f=cv2.flip(f,1)
    rgb=cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    op=hand_detect.process(rgb)   
    
    #tool bar
    cv2.rectangle(f,(80,10),(620,80),(0,0,0),2)
    cv2.line(f,(170,10),(170,80),(0,0,0),2)
    cv2.line(f,(260,10),(260,80),(0,0,0),2)
    cv2.line(f,(350,10),(350,80),(0,0,0),2)
    cv2.line(f,(440,10),(440,80),(0,0,0),2)
    cv2.line(f,(530,10),(530,80),(0,0,0),2)
    cv2.circle(f,(125,45),25,(0,0,0),4)
    cv2.rectangle(f,(190,30),(240,60),(0,0,0),4)
    cv2.line(f,(280,60),(330,30),(0,0,0),4)
    cv2.line(f,(360,70),(390,30),(0,0,0),4)
    cv2.line(f,(390,30),(390,70),(0,0,0),4)
    cv2.line(f,(390,70),(430,30),(0,0,0),4)
    cv2.line(f,(430,30),(430,50),(0,0,0),4)
    cv2.rectangle(f,(458,61),(512,29),(0,0,0),2)
    cv2.rectangle(f,(460,60),(510,30),(0,255,255),-1)
    cv2.line(f,(495,30),(495,60),(0,0,0),4)
    cv2.putText(f,"clear",(535,50),cv2.FONT_HERSHEY_COMPLEX,0.9,(0,0,0),2,cv2.LINE_8,False)
    
    #color bar
    cv2.rectangle(f,(600,110),(650,590),(0,0,0),2)
    cv2.line(f,(600,190),(650,190),(0,0,0),2)
    cv2.line(f,(600,270),(650,270),(0,0,0),2)
    cv2.line(f,(600,350),(650,350),(0,0,0),2)
    cv2.line(f,(600,430),(650,430),(0,0,0),2)
    cv2.line(f,(600,510),(650,510),(0,0,0),2)
    cv2.rectangle(f,(602,112),(648,188),(0,0,255),-1)
    cv2.rectangle(f,(602,192),(648,268),(255,0,0),-1)
    cv2.rectangle(f,(602,272),(648,348),(0,255,0),-1)
    cv2.rectangle(f,(602,352),(648,428),(0,255,255),-1)
    cv2.rectangle(f,(602,432),(648,508),(255,255,255),-1)
    cv2.rectangle(f,(602,512),(648,588),(0,0,0),-1)
    
   
    if r==True:
        if op.multi_hand_landmarks:
            for i in op.multi_hand_landmarks:
                draw.draw_landmarks(f,i,hands.HAND_CONNECTIONS)
                x,y=int(i.landmark[8].x*700),int(i.landmark[8].y*700)
                
                if (x < max_x_tool) & (y < max_y_tool) & (x > left_margin):
                    
                    if time_intialised:
                        current_time=time.time()
                        time_intialised=False
                        
                    p_time=time.time()
                    cv2.circle(f,(x,y),circle_rad,(0,255,255),2)
                    circle_rad-=1
                    
                    if p_time-current_time > 0.8:
                        curr_tool=get_tool(x)
                        time_intialised=True
                        circle_rad=40
                
                elif (min_x_color < x < max_x_color) & (min_y_color < y < max_y_color) :
                    
                    if time_intialised:
                        current_time=time.time()
                        time_intialised=False
                        
                    p_time=time.time()
                    cv2.circle(f,(x,y),circle_rad,(255,255,255),2)
                    circle_rad-=1
                    
                    if p_time-current_time > 0.8:
                        curr_color=get_color(y)
                        time_intialised=True
                        circle_rad=40
                        
                elif curr_tool == "circle":
                    
                    xi,yi=int(i.landmark[12].x*700),int(i.landmark[12].y*700)
                    y9=int(i.landmark[9].y*700)
                    
                    if index_raised(yi,y9):
                        
                        if not(mode):
                            mode=True
                            xii,yii=x,y
                        
                        cv2.circle(f,(xii,yii),int(((xii-x)**2 + (yii-y)**2)**0.5),curr_color,2)
                    
                    else:
                        if mode==True:
                            mode=False
                            cv2.circle(mask,(xii,yii),int(((xii-x)**2 + (yii-y)**2)**0.5),curr_color,2)
                            
                elif curr_tool=="rectangle":
                    
                    xi,yi=int(i.landmark[12].x*700),int(i.landmark[12].y*700)
                    y9=int(i.landmark[9].y*700)
                    
                    if index_raised(yi,y9):
                        
                        if not(mode):
                            mode=True
                            xii,yii=x,y
                        
                        cv2.rectangle(f,(xii,yii),(x,y),curr_color,2)
                    
                    else:
                        if mode==True:
                            mode=False
                            cv2.rectangle(mask,(xii,yii),(x,y),curr_color,2)
                            
                elif curr_tool=="line":
                    
                    xi,yi=int(i.landmark[12].x*700),int(i.landmark[12].y*700)
                    y9=int(i.landmark[9].y*700)
                    
                    if index_raised(yi,y9):
                        
                        if not(mode):
                            mode=True
                            xii,yii=x,y
                        
                        cv2.line(f,(xii,yii),(x,y),curr_color,2)
                    
                    else:
                        if mode==True:
                            mode=False
                            cv2.line(mask,(xii,yii),(x,y),curr_color,2)
                            
                elif curr_tool=="erase":
                    
                    xi, yi = int(i.landmark[12].x*700), int(i.landmark[12].y*700)
                    y9  = int(i.landmark[9].y*700)

                    if index_raised(yi, y9):
                        cv2.circle(f, (x, y), 30, (0,0,0), -1)
                        cv2.circle(mask, (x, y), 30, 255, -1)
                
                elif curr_tool=="draw":
                    
                    xi, yi = int(i.landmark[12].x*700), int(i.landmark[12].y*700)
                    y9  = int(i.landmark[9].y*700)

                    if index_raised(yi, y9):
                        cv2.line(mask, (prevx, prevy),(x,y),curr_color,2)
                        prevx,prevy=x,y
                    else:
                        prevx=x
                        prevy=y
                        
                elif curr_tool=="clear":
                    cv2.rectangle(mask,(0,0),(700,700),255,-1)
                
           
        op = cv2.bitwise_and(f, f, mask=mask)
        f[:, :, 1] = op[:, :, 1]
        f[:, :, 2] = op[:, :, 2]
        
        cv2.putText(f, "mode: "+curr_tool, (280,110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, curr_color, 2)
        cv2.putText(f, "color: "+color_name(curr_color), (280,140), cv2.FONT_HERSHEY_SIMPLEX, 0.8, curr_color, 2)
        
        cv2.imshow("Canvas",mask)    
        cv2.imshow("AirCanvas",f)
        if cv2.waitKey(25) & 0xff==ord("p"):break
    else: break
        
cap.release()
cv2.destroyAllWindows()